In [1]:
import copernicusmarine as cm
import numpy as np
import netCDF4 as nc
import xarray as xr

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
cm.subset(
    dataset_id="c3s_obs-sl_glo_phy-ssh_my_twosat-l4-duacs-0.25deg_P1D",
    dataset_version="202112",
    variables=["adt"],
    minimum_longitude=-101.0,
    maximum_longitude=-50.0,
    minimum_latitude=14.0,
    maximum_latitude=52.0,
    start_datetime="1993-01-01T00:00:00",
    end_datetime="2023-06-07T00:00:00",
    output_filename="data_copernicus_adt_1993_2023.nc"
)

#sl_glo_phy_obj = netCDF4.Dataset('./data_copernicus_adt_1993_2023.nc')

In [34]:
sl_glo_phy_xr = xr.open_dataset('data_copernicus_adt_1993_2023.nc')
sl_glo_phy_xr_df = sl_glo_phy_xr.to_dataframe()

In [3]:
# Open the NetCDF file
import pandas as pd
file_path = 'data_copernicus_adt_1993_2023.nc'
nc_file = nc.Dataset(file_path)

/var/folders/3m/g7qw5r_50tj5yh0jj5dphfj00000gp/T/ipykernel_84959/3584571726.py:6: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  time = nc_file.variables['time'][:]
/var/folders/3m/g7qw5r_50tj5yh0jj5dphfj00000gp/T/ipykernel_84959/3584571726.py:6: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  time = nc_file.variables['time'][:]


ValueError: All arrays must be of the same length

In [4]:
print(nc_file)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    Metadata_Conventions: Unidata Dataset Discovery v1.0
    cdm_data_type: Grid
    comment: Sea Surface Height measured by Altimetry and derived variables
    contact: http://climate.copernicus.eu/c3s-user-service-desk
    creator_email: http://climate.copernicus.eu/c3s-user-service-desk
    creator_name: Copernicus Climate Change Service (C3S)
    creator_url: http://climate.copernicus.eu
    date_created: 2021-07-09T08:53:57Z
    date_issued: 2021-07-09T08:53:57Z
    date_modified: 2021-07-09T08:53:57Z
    geospatial_lat_max: 89.875
    geospatial_lat_min: -89.875
    geospatial_lat_resolution: 0.25
    geospatial_lat_units: degrees_north
    geospatial_lon_max: 179.875
    geospatial_lon_min: -179.875
    geospatial_lon_resolution: 0.25
    geospatial_lon_units: degrees_east
    geospatial_vertical_max: 0.0
    geospatial_vertical_min: 0.0
    geospatial_vertical_positive:

In [5]:
print(nc_file.variables.keys()) # get all variable names

dict_keys(['adt', 'latitude', 'longitude', 'time'])


In [10]:
for d in nc_file.dimensions.items():
    print(d)

('time', <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 11115)
('latitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 152)
('longitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 204)


In [16]:
time, longitude, latitude = nc_file.variables['time'][:], nc_file.variables['longitude'][:], nc_file.variables['latitude'][:]

/var/folders/3m/g7qw5r_50tj5yh0jj5dphfj00000gp/T/ipykernel_84959/2488558443.py:1: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  time, longitude, latitude = nc_file.variables['time'][:], nc_file.variables['longitude'][:], nc_file.variables['latitude'][:]
/var/folders/3m/g7qw5r_50tj5yh0jj5dphfj00000gp/T/ipykernel_84959/2488558443.py:1: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  time, longitude, latitude = nc_file.variables['time'][:], nc_file.variables['longitude'][:], nc_file.variables['latitude'][:]


In [21]:
adt = nc_file.variables['adt'][:]

In [25]:
adt[0,0,0]

0.78

In [26]:
adt_flat = adt.reshape((-1,))

In [30]:
latitude.shape

(152,)

In [39]:
ntime, nlat, nlon = len(time), len(latitude), len(longitude)

time_list, lat_list, lon_list, adt_list = [], [], [], []
for x in range(ntime):
    for y in range(nlat):
        for z in range(nlon):
            time_list.append(time[x])
            lat_list.append(latitude[y])
            lon_list.append(longitude[z])
            adt_list.append(adt[x,y,z])

df = pd.DataFrame({'Time': time_list, 'Latitude': lat_list, 'Longitude': lon_list, 'Adt': adt_list})

In [40]:
df.head()

,Time,Latitude,Longitude,Adt
0,15706.0,14.125,-100.875,0.78
1,15706.0,14.125,-100.625,0.8006
2,15706.0,14.125,-100.375,0.8169
3,15706.0,14.125,-100.125,0.8166
4,15706.0,14.125,-99.875,0.8135


In [42]:
df.to_csv('./data_copernicus_adt_1993_2023.csv')

In [ ]:

time = nc_file.variables['time'][:]
longitude = nc_file.variables['longitude'][:]
latitude = nc_file.variables['latitude'][:]
adt = nc_file.variables['adt'][:].flatten()

data = {'time': time, 'longitude': longitude, 'latitude': latitude, 'adt': adt}
df = pd.DataFrame(data)

# Close the NetCDF file
nc_file.close()

In [45]:
df[df['Adt']==max(df['Adt'])]

,Time,Latitude,Longitude,Adt
121290801,19617.0,36.625,-62.625,1.6953


In [46]:
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

df_plot = df[df['Longitude']==-62.625 & df['Latitude']==36.625]
plt.plot(df_plot['Time'], df_plot['Atd'])

In [ ]:
from mpl_toolkits.basemap import Basemap

L_df = sl_glo_phy_xr_df[['latitude','longitude']]
L_df.columns = ['Longitude', 'Latitude']
plt.figure(figsize=(12, 9))

m = Basemap(projection='merc', llcrnrlat=24, urcrnrlat=38, llcrnrlon=-95, urcrnrlon=-75, resolution='i')

m.drawcoastlines()
m.drawcountries()
m.drawstates()

x, y = m(L_df['Longitude'].values, L_df['Latitude'].values)

m.scatter(x, y, s=50, c='red', marker='o', label='Stations', zorder=5)

plt.title('Location of Stations on U.S. Southeast and Gulf Coast')
plt.legend(loc='lower left')
plt.show()